## 패키지 설치 및 구글 드라이브 연동

In [1]:
# 구글 드라이브와 연결
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [2]:
# 의존성 패키지 설치
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 57.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.1 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=94cdea87279ded3b4d76a9d7ffaf6c686120f0f11e8449f3d1d676f2ca401580
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

## GPT 입력값 만들기

GPT 모델 입력값을 만들려면 Byte-level Byte Pair Encoding 어휘집합 구축 결과(`vocab.json`, `merges.txt`)가 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 있어야 합니다. 다음을 수행해 이미 만들어 놓은 BBPE 어휘집합을 포함한 GPT 토크나이저를 `tokenizer_gpt`라는 변수로 선언합니다.


In [3]:
# GPT 토크나이저 선언
from transformers import GPT2Tokenizer
tokenizer_gpt=GPT2Tokenizer.from_pretrained('/gdrive/MyDrive/nlpbook/bbpe')
tokenizer_gpt.pad_token="[PAD]"

file /gdrive/MyDrive/nlpbook/bbpe/config.json not found


In [4]:
# GPT 토크나이저로 토큰화하기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences=[tokenizer_gpt.tokenize(sentence) for sentence in sentences]

In [5]:
# GPT 토크나이저 토큰화 결과
# 알 수 없는 문자열인 이유: GPT모델은 바이트 기준 BPE를 적용
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

In [6]:
# GPT 모델 입력 만들기
batch_inputs=tokenizer_gpt(
    sentences,
    padding='max_length',    # 문장의 최대 길이에 맞춰 패딩
    max_length=12,           # 문장의 토큰 기준 최대 길이
    truncation=True          # 문장 잘림 옵션 허용
)

batch_inputs 내용 확인 : 2가지 입력값 생성

In [7]:
batch_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [8]:
batch_inputs['input_ids']

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

In [9]:
# 1: 일반 토큰이 자리한 곳 / 0: 패딩 토큰이 자리한 곳
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

## BERT 입력값 만들기

In [10]:
# BERT 토크나이저 선언
from transformers import BertTokenizer
tokenizer_bert=BertTokenizer.from_pretrained(
    '/gdrive/MyDrive/nlpbook/wordpiece',
    do_lower_case=False
)

file /gdrive/MyDrive/nlpbook/wordpiece/config.json not found


In [11]:
# BERT 토크나이저로 토큰화하기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]

tokenized_sentences=[tokenizer_bert.tokenize(sentence) for sentence in sentences]

In [12]:
# BERT 토크나이저 토큰화 결과
# '##'의 의미: 해당 토큰이 어절의 시작이 아님을 나타낸다.
tokenized_sentences

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영화',
  '##줄',
  '.',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['별루', '였다', '.', '.']]

In [14]:
# BERT 모델 입력 만들기
batch_inputs=tokenizer_bert(
    sentences,
    padding='max_length',
    max_length=12,
    truncation=True
)

batch_inputs 내용 확인

In [15]:
batch_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [16]:
# 2: [CLS] - 문장의 시작 토큰 / 3: [SEP] - 문장의 끝 토큰
batch_inputs['input_ids']

[[2, 620, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1506, 16, 3],
 [2, 3274, 9508, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

In [18]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

In [17]:
# 세그먼트 정보
batch_inputs['token_type_ids']

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

## 미리 학습된 언어 모델

1. 언어모델: 단어 시퀀스에 확률을 부여하는 모델
    - n개 단어가 동시에 나타날 결합확률
    - 조건부확률
    - 첫번째단어등장 → 첫단어등장 후 두번째단어 등장 → 첫단어, 두번째 단어 등장 후 세번째단어 등장
    - 전체단어시퀀스가 나타날 확률은 이전단어들이 주어졌을 때 다음단어가 등장할 확률의 연쇄
    - 이전 단어들이 주어졌을 때 다음 단어가 나타날 확률을 부여하는 모델
    1. 순방향 언어 모델: 문장 앞부터 뒤로, 사람이 이해하는 순서대로 계산하는 모델(GPT,ELMo모델)
    2. 역방향 언어 모델: 문장 뒤부터 앞으로 계산(ELMo모델-순방향&역방향 모두 활용 기법)

- $P(w|context)$ : 넓은 의미의 언어모델    
    3. 마스크 언어 모델: 학습 대상 문장에 빈칸 만든 후 빈칸에 올 단어로 적절한 단어 무엇인지 분류(BERT모델)
        - 문장 전체의 맥락을 참고할 수 있다.(양방향 성질)
    4. 스킵-그램 모델: 어떤 단어 앞뒤에 특정범위 지정한 후 범위 내에 어떤 단어 올지 분류(Word2Vec모델)
        - context로 설정한 단어 주변에 어떤 단어들이 분포해있는지를 학습
    5. 유용성(예시)
        - 기계 번역: P(?|You can't be free from death)
        - 문법 교정: P(두시 삼십 이분) > P(이시 서른 두분)
        - 문장 생성: P(?|발 없는 말이)
    6. 언어 모델이 주목받는 이유
        - 데이터 제작 비용
        - 트랜스퍼 러닝